# 6. Regime Switching Models

The purpose of this section is to identify **latent structural regimes** in the
time series (e.g., periods of high growth vs. low or negative growth). A Hidden
Markov Model (HMM) is used to classify each year into an unobserved “state”
based on the pattern of year-to-year changes.

Why HMM?

- It detects **unobserved states** that govern the system dynamics.
- Captures transitions between **stable**, **volatile**, **growth**, or **decline** regimes.
- Useful for identifying turning points and structural phases.

Workflow:
1. Compute the **first difference** (yearly growth/decline).
2. Fit a **Gaussian HMM** with 2 regimes.
3. Predict regime for each year.
4. Visualize regime-separated trends.
5. Reorder regimes by average level to ensure interpretable labeling.
### Interpretation of Regime Switching Results

- **Regime 0 (low-level / low-growth period):**
  Characterized by lower values of the variable or weaker growth.
  Typically represents stabilization or stagnation phases.

- **Regime 1 (high-level / high-growth period):**
  Shows higher values or stronger year-to-year increases.
  Often corresponds to periods of rapid development, expansion, or policy shifts.

Transitions between regimes may align with:
- economic cycles
- major zoning or planning policy changes
- infrastructure investments
- market shocks
- long-term development phases in Charlotte

HMM helps reveal **hidden structural phases** that simple trend plots would not show.



In [ ]:
# ============================================================
# Regime Switching Model — Hidden Markov Model (HMM)
# ============================================================

# ------------------------------------------------------------
# 1. Prepare data for HMM
# ------------------------------------------------------------
# HMM requires a 2D feature array.
# We use first differences (year-over-year growth) because:
#   - differences are closer to stationary
#   - regimes are typically based on growth behavior
#   - avoids fitting HMM directly on a trending nonstationary series
y = ts.values
dy = np.diff(y)  # year-to-year change

# Reshape into 2D array for HMM input
X = dy.reshape(-1, 1)

# ------------------------------------------------------------
# 2. Fit a 2-state Gaussian HMM
# ------------------------------------------------------------
# n_components = 2 → classify into 2 hidden regimes
# covariance_type='full' → flexible covariance structure
# n_iter=500 → allow enough iterations for convergence
hmm = GaussianHMM(n_components=2,
                  covariance_type="full",
                  n_iter=500,
                  random_state=42)

hmm.fit(X)

# Predict hidden state for each year except the first (because differencing)
states = hmm.predict(X)

# Align years with states
years = ABT['year'].iloc[1:].values  # drop first year to match dy length

# Build a DataFrame of results
regime_df = gpd.GeoDataFrame({
    "year": years,
    f"{measure}_mean": y[1:],   # drop first entry to align length
    f"{measure}_growth": dy,
    "regime": states
})

# ------------------------------------------------------------
# 3. Plot regime-separated series
# ------------------------------------------------------------
plt.figure(figsize=(14, 6))

# Plot the series separately for each regime
for r in sorted(regime_df['regime'].unique()):
    subset = regime_df[regime_df['regime'] == r]
    plt.plot(subset['year'], subset[f"{measure}_mean"],
             'o-', linewidth=2,
             label=f"Regime {r}")

plt.title(f"HMM Regime Classification for {measure}_mean")
plt.xlabel("Year")
plt.ylabel(f"{measure}_mean")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

# ------------------------------------------------------------
# 4. Reorder regime labels based on average level
# ------------------------------------------------------------
# Meaning:
#   Lower average → label as Regime 0
#   Higher average → label as Regime 1
# This ensures interpretability regardless of HMM's arbitrary labeling.
regime_means = regime_df.groupby('regime')[f"{measure}_mean"].mean()

# Sort regimes by mean level
regime_map = {old_label: new_label
              for new_label, old_label in enumerate(regime_means.sort_values().index)}

# Assign ordered labels
regime_df["regime_ordered"] = regime_df["regime"].map(regime_map)

# ------------------------------------------------------------
# 5. (Optional) Plot reordered regimes for interpretation
# ------------------------------------------------------------
plt.figure(figsize=(14, 6))

for r in sorted(regime_df['regime_ordered'].unique()):
    subset = regime_df[regime_df["regime_ordered"] == r]
    plt.plot(subset["year"], subset[f"{measure}_mean"],
             'o-', linewidth=2,
             label=f"Ordered Regime {r}")

plt.title(f"Ordered Regimes for {measure}_mean (HMM)")
plt.xlabel("Year")
plt.ylabel(f"{measure}_mean")
plt.grid(alpha=0.3)
plt.legend()
plt.show()

In [ ]:
o	Apply Markov-Switching AR to see if SUM shifted regimes (e.g., sprawl → densification).
4.	Optional
o	Try TVAR or nonlinear AR if evidence of nonlinearity.
o	Use wavelets for describing multi-scale effects, not prediction.



__4. Visualization__


In [ ]:
sns.set_style("whitegrid")
plt.rcParams.update({
    "font.size": 14,
    "axes.titlesize": 18,
    "axes.labelsize": 16,
    "legend.fontsize": 13
})

fig, ax = plt.subplots(figsize=(14, 7))

# Observed series
ax.plot(ABT['year'], ABT[f'{measure}_mean'], 
        marker='o', markersize=6, color='steelblue', linewidth=2, label='Observed')

# LOWESS trend
ax.plot(smoothed[:, 0], smoothed[:, 1], 
        color='crimson', linestyle='--', linewidth=3, label='LOWESS Trend')

# Wavelet trend
ax.plot(ABT['year'], ABT['wavelet_trend'],
        color='goldenrod', linestyle=':', linewidth=2, label='Wavelet (Long-term trend)')


# Shade regimes detected by HMM (group consecutive same-state years)
current_state = hidden_states[0]
start_idx = 0
for i in range(1, len(hidden_states)):
    if hidden_states[i] != current_state or i == len(hidden_states) - 1:
        end_year = ABT['year'].iloc[i] if i == len(hidden_states) - 1 else ABT['year'].iloc[i - 1]
        ax.axvspan(ABT['year'].iloc[start_idx] - 0.4, end_year + 0.4,
                   color='lightgreen' if current_state == 1 else 'lightgray', alpha=0.25)
        start_idx = i
        current_state = hidden_states[i]

# ---- Y-axis range: use true min/max of the series ----
y_min = ABT[f'{measure}_mean'].min()
y_max = ABT[f'{measure}_mean'].max()
y_pad = (y_max - y_min) * 0.1   # 10% padding for visual spacing
ax.set_ylim(y_min - y_pad, y_max + y_pad)

# Labels and styling
ax.set_title(f"{measure} (County-Level Mean) – Trend, Wavelet, and Hidden States", pad=20, weight='bold')
ax.set_xlabel('Year')
ax.set_ylabel(f"{measure} Mean Value")
ax.legend(loc='upper left', frameon=True)
sns.despine()
plt.tight_layout()
plt.show()

__Rolling Statistics__

In [ ]:
window = 5  # 5-year rolling window
ABT['rolling_mean'] = ABT[f'{measure}_mean'].rolling(window).mean()
ABT['rolling_std'] = ABT[f'{measure}_mean'].rolling(window).std()

plt.figure(figsize=(10,5))
plt.plot(ABT['year'], ABT[f'{measure}_mean'], label='Original', color='steelblue')
plt.plot(ABT['year'], ABT['rolling_mean'], label='Rolling Mean (5y)', color='red')
plt.plot(ABT['year'], ABT['rolling_std'], label='Rolling Std (5y)', color='orange')
plt.legend(); plt.show()


__Autoregressive Distributed Lag (ARDL)__

In [ ]:
y = ABT[f'{measure}_mean']
model_ardl = ARDL(y, lags=3)      # try 1–3 lags
fit_ardl = model_ardl.fit()
print(fit_ardl.summary())
fit_ardl.plot_diagnostics()

__Generalized Additive Model (GAM)__

In [ ]:
# Fit the model
fit_hw = ExponentialSmoothing(y, trend='add', damped_trend=True).fit()

# Forecast next 5 years
forecast_steps = 5
forecast = fit_hw.forecast(forecast_steps)

# Build the time index for forecasts
future_years = np.arange(ABT['year'].max() + 1, ABT['year'].max() + 1 + forecast_steps)

# Plot observed, fitted, and forecasted
plt.figure(figsize=(10, 5))
plt.plot(ABT['year'], y, label='Observed', color='steelblue', marker='o')
plt.plot(ABT['year'], fit_hw.fittedvalues, label='Fitted (in-sample)', color='crimson')
plt.plot(future_years, forecast, label='Forecast (next 5 years)', color='goldenrod', linestyle='--', marker='o')
plt.title("Holt-Winters Damped Trend Forecast", fontsize=14)
plt.xlabel("Year")
plt.ylabel(f"{measure} Mean")
plt.legend()
plt.grid(True)
plt.show()

__Autoregressive Conditional Heteroskedasticity (ARCH/GARCH)__

In [ ]:
garch = arch_model(y, vol='GARCH', p=1, q=1)
fit_garch = garch.fit(disp='off')
fit_garch.plot(annualize='D'); plt.show()

__Permutation Entropy / Complexity__

In [ ]:
pe = ant.perm_entropy(y, normalize=True)
print("Permutation Entropy:", pe)

In [ ]:
!jupyter nbconvert --to html --no-input time_series_univariate.ipynb --output ../../../../output/Notebook_Outputs/time_series_univariate.html